In [ ]:
# ---
# jupyter:
#   jupytext:
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.16.3
#   kernelspec:
#     display_name: Python 3
#     language: python
#     name: python3
# ---

# # Crypto AI Agent
# Fetches Bitcoin data from CoinGecko, calculates technical indicators, trains a Random Forest model, and predicts Buy/Sell actions.

# ## Imports
import pandas as pd
import numpy as np
from pycoingecko import CoinGeckoAPI
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
import logging
import warnings
warnings.filterwarnings('ignore')

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ## Fetch Crypto Data
def fetch_crypto_data(use_mock=False):
    """
    Fetch Bitcoin OHLC and volume data from CoinGecko.
    Fallback to mock data if API fails.
    """
    if use_mock:
        logging.info("Using mock data")
        return pd.read_csv('mock_crypto_data.csv')
    
    try:
        cg = CoinGeckoAPI()
        # Fetch OHLC data (Open, High, Low, Close)
        ohlc = cg.get_coin_ohlc_by_id(id='bitcoin', vs_currency='usd', days='30')
        df = pd.DataFrame(ohlc, columns=['timestamp', 'open', 'high', 'low', 'close'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        
        # Fetch volume data
        market_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days='30')
        df['volume'] = [v for t, v in market_data['total_volumes']][:len(df)]
        
        logging.info(f"Successfully fetched {len(df)} rows from CoinGecko")
        return df
    except Exception as e:
        logging.error(f"CoinGecko failed: {e}")
        logging.info("Falling back to mock data")
        return pd.read_csv('mock_crypto_data.csv')

# ## Calculate Technical Indicators
def calculate_indicators(df):
    """
    Calculate SMA, RSI, MACD, and Bollinger Bands.
    """
    # Simple Moving Averages
    df['sma_10'] = df['close'].rolling(window=10).mean()
    df['sma_50'] = df['close'].rolling(window=50).mean()
    
    # RSI
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # MACD
    ema_12 = df['close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = ema_12 - ema_26
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
    
    # Bollinger Bands
    df['bb_middle'] = df['close'].rolling(window=20).mean()
    df['bb_std'] = df['close'].rolling(window=20).std()
    df['bb_upper'] = df['bb_middle'] + (df['bb_std'] * 2)
    df['bb_lower'] = df['bb_middle'] - (df['bb_std'] * 2)
    
    return df

# ## Prepare Data for Model
def prepare_data(df):
    """
    Create features and target for the model.
    Target: 1 (Buy) if next day's close > current close, else 0 (Sell).
    """
    df = calculate_indicators(df)
    
    # Features
    features = ['sma_10', 'sma_50', 'rsi', 'close', 'volume', 'macd', 'macd_signal', 'bb_upper', 'bb_lower']
    df = df.dropna()
    
    # Target
    df['target'] = (df['close'].shift(-1) > df['close']).astype(int)
    
    X = df[features]
    y = df['target']
    
    return X, y, df

# ## Train Model
def train_model(X, y):
    """
    Train a Random Forest Classifier.
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    logging.info(f"Model accuracy: {accuracy:.2f}")
    
    return model

# ## Predict
def predict(model, latest_data):
    """
    Predict Buy/Sell for the latest data point.
    """
    X, _, _ = prepare_data(latest_data)
    if not X.empty:
        latest_features = X.iloc[-1].values.reshape(1, -1)
        prediction = model.predict(latest_features)[0]
        probability = model.predict_proba(latest_features)[0][prediction]
        return prediction, probability
    return None, None

# ## Main Execution
def main():
    # Fetch data
    data = fetch_crypto_data(use_mock=False)
    latest_data = fetch_crypto_data(use_mock=False)
    
    # Prepare data
    X, y, df = prepare_data(data)
    
    # Train model
    model = train_model(X, y)
    
    # Save model
    joblib.dump(model, 'crypto_model.pkl')
    logging.info("Model saved as crypto_model.pkl")
    
    # Predict
    prediction, probability = predict(model, latest_data)
    if prediction is not None:
        action = "Buy" if prediction == 1 else "Sell"
        latest_price = latest_data['close'].iloc[-1]
        logging.info(f"Latest Price: ${latest_price:,.2f}")
        logging.info(f"Recommended Action: {action} (Confidence: {probability:.2f})")
    else:
        logging.error("Prediction failed: No valid data")

if __name__ == "__main__":
    main()

# ## Notes
# - Data source: CoinGecko (reliable as of April 2025).
# - Features: SMA (10, 50), RSI, MACD, Bollinger Bands, close, volume.
# - Model: Random Forest Classifier.
# - Output: Buy/Sell prediction with confidence score.


[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


INFO:__main__:Fetching data from Kraken for BTC/USD
ERROR:__main__:Kraken failed for BTC/USD: kraken GET https://api.kraken.com/0/public/Assets
INFO:__main__:Trying Binance for BTC/USDT
ERROR:__main__:Binance failed for BTC/USDT: binance GET https://api.binance.com/api/v3/exchangeInfo
INFO:__main__:Model trained and saved as crypto_model.pkl
INFO:__main__:Fetching data from Kraken for BTC/USD
ERROR:__main__:Kraken failed for BTC/USD: kraken GET https://api.kraken.com/0/public/Assets
INFO:__main__:Trying Binance for BTC/USDT
ERROR:__main__:Binance failed for BTC/USDT: binance GET https://api.binance.com/api/v3/exchangeInfo
INFO:__main__:Predicted action: Buy at price: $81891.58


Latest Price: $81891.58
Recommended Action: Buy


Exported Colab notebook to a local project folder, preparing code for Flask integration.